In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import requests
import json

# Import API key
from api_keys import api_key

In [7]:
oscar_data= pd.read_csv("Resources/oscar_data.csv")
oscar_data.head()

,Year,Host,Most Nominations,Most Wins,Best Picture,Nominee 1,Nominee 2,Nominee 3,Nominee 4,Nominee 5,...,Best Costume Design (Black and White),Nominee 1.31,Nominee 2.31,Nominee 3.30,Nominee 4.30,Best Costume Design (Color),Nominee 1.32,Nominee 2.32,Nominee 3.31,Nominee 4.31
0,1927/28,Douglas Fairbanks,Seventh Heaven (5),Seventh Heaven AND Sunrise: A Song of Two Huma...,Wings,The Racket,Seventh Heaven,Unique and Artistic Production- Sunrise: A Son...,Chang: A Drama of the Wilderness,The Crowd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1928/29,William C. DeMille,In Old Arizona AND The Patriot (5),None (No Film Won More Than One Award),The Broadway Melody,Alibi,Hollywood Revue,In Old Arizona,The Patriot,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1929/30,Conrad Nagel,The Love Parade (6),All Quiet on the Western Front AND The Big Hou...,All Quiet on the Western Front,The Big House,Disraeli,The Divorcee,The Love Parade,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1930/31,Lawrence Grant,Cimarron (7),Cimarron (3),Cimarron,East Lynne,The Front Page,Skippy,Trader Horn,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1931/32,Conrad Nagel,Arrowsmith AND The Champ (4),Bad Girl AND The Champ (2),Grand Hotel,Arrowsmith,Bad Girl,The Champ,Five Star Final,One Hour with You,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
nominees = oscar_data[["Best Picture", "Nominee 1", "Nominee 2", "Nominee 3", "Nominee 4"]]
nominees

,Best Picture,Nominee 1,Nominee 2,Nominee 3,Nominee 4
0,Wings,The Racket,Seventh Heaven,Unique and Artistic Production- Sunrise: A Son...,Chang: A Drama of the Wilderness
1,The Broadway Melody,Alibi,Hollywood Revue,In Old Arizona,The Patriot
2,All Quiet on the Western Front,The Big House,Disraeli,The Divorcee,The Love Parade
3,Cimarron,East Lynne,The Front Page,Skippy,Trader Horn
4,Grand Hotel,Arrowsmith,Bad Girl,The Champ,Five Star Final
...,...,...,...,...,...
91,Parasite,1917,Ford v Ferrari,The Irishman,Jojo Rabbit
92,Nomadland,The Father,Judas and the Black Messiah,Mank,Minari
93,CODA,Belfast,Don't Look Up,Drive My Car,Dune
94,Everything Everywhere All At Once,All Quiet on the Western Front,Avatar: The Way of Water,The Banshees of Inisherin,Elvis


In [ ]:
url = "https://api.themoviedb.org/3"

startdate = "2022-11-01"
enddate = "2022-12-01"

response = requests.get(f'{url}/discover/movie?include_adult=false&include_video=false&primary_release_date.gte={startdate}&primary_release_date.lte={enddate}&with_original_language=en&api_key={api_key}').json()

In [36]:
responseid = {'Title' : [],'ID' : [], 'Vote Average' : []}

for i in range(10):
    responseid['Title'].append(response["results"][i]["title"])
    responseid['ID'].append(response["results"][i]["id"])
    responseid['Vote Average'].append(response["results"][i]["vote_average"])

moviedata = pd.DataFrame(responseid)
moviedata

,Title,ID,Vote Average
0,Black Panther: Wakanda Forever,505642,7.142
1,Glass Onion: A Knives Out Mystery,661374,7.052
2,Disenchanted,338958,6.792
3,Lamborghini: The Man Behind the Legend,457232,6.460
4,Enola Holmes 2,829280,7.420
5,The Menu,593643,7.190
6,R.I.P.D. 2: Rise of the Damned,1013860,6.360
7,Violent Night,899112,7.383
8,The Fabelmans,804095,7.651
9,Roald Dahl's Matilda the Musical,668482,6.775


In [19]:
title = []
vote = []
release_date = []

for i in range(10):
    movie_id = i

    movie_response = requests.get(f'{url}/movie/{movie_id}?api_key={api_key}').json()
    
    try:
        print(f'Title = {movie_response["original_title"]}')
        title.append(movie_response["original_title"])
    except: KeyError

    try:
        print(f'Average Vote = {movie_response["vote_average"]}')
        vote.append(movie_response["vote_average"])
    except: KeyError

    try:
        print(f'Release Date = {movie_response["release_date"]}')
        release_date.append(movie_response["release_date"])
    except: KeyError

Title = Ariel
Average Vote = 7.092
Release Date = 1988-10-21
Title = Varjoja paratiisissa
Average Vote = 7.302
Release Date = 1986-10-17
Title = Four Rooms
Average Vote = 5.82
Release Date = 1995-12-09
Title = Judgment Night
Average Vote = 6.527
Release Date = 1993-10-15


KeyboardInterrupt: 

In [15]:
movie_id = 1022796

movie_response = requests.get(f'{url}/movie/{movie_id}?api_key={api_key}').json()

print(json.dumps(movie_response, indent=4, sort_keys=True))
print(f'Average Vote = {movie_response["vote_average"]}')
print(f'Release Date = {movie_response["release_date"]}')

{
    "adult": false,
    "backdrop_path": "/ehumsuIBbgAe1hg343oszCLrAfI.jpg",
    "belongs_to_collection": null,
    "budget": 175000000,
    "genres": [
        {
            "id": 16,
            "name": "Animation"
        },
        {
            "id": 10751,
            "name": "Family"
        },
        {
            "id": 14,
            "name": "Fantasy"
        },
        {
            "id": 12,
            "name": "Adventure"
        }
    ],
    "homepage": "https://movies.disney.com/wish",
    "id": 1022796,
    "imdb_id": "tt11304740",
    "original_language": "en",
    "original_title": "Wish",
    "overview": "Asha, a sharp-witted idealist, makes a wish so powerful that it is answered by a cosmic force \u2013 a little ball of boundless energy called Star. Together, Asha and Star confront a most formidable foe - the ruler of Rosas, King Magnifico - to save her community and prove that when the will of one courageous human connects with the magic of the stars, wondrous t

In [17]:
title = []
vote = []
release_date = []

for i in range(10):
    movie_id = i

    movie_response = requests.get(f'{url}/movie/{movie_id}?api_key={api_key}').json()
    
    try:
        print(f'Title = {movie_response["original_title"]}')
        title.append(movie_response["original_title"])
    except: KeyError

    try:
        print(f'Average Vote = {movie_response["vote_average"]}')
        vote.append(movie_response["vote_average"])
    except: KeyError

    try:
        print(f'Release Date = {movie_response["release_date"]}')
        release_date.append(movie_response["release_date"])
    except: KeyError

Title = Ariel
Average Vote = 7.092
Release Date = 1988-10-21
Title = Varjoja paratiisissa
Average Vote = 7.302
Release Date = 1986-10-17
Title = Four Rooms
Average Vote = 5.82
Release Date = 1995-12-09
Title = Judgment Night
Average Vote = 6.527
Release Date = 1993-10-15
Title = Life in Loops (A Megacities RMX)
Average Vote = 7.5
Release Date = 2006-01-01
Title = Sonntag im August
Average Vote = 7.135
Release Date = 2004-09-02


In [18]:
df = pd.DataFrame({"Title" : title,
                   "Vote" : vote,
                   "Release Date" : release_date})
df

,Title,Vote,Release Date
0,Ariel,7.092,1988-10-21
1,Varjoja paratiisissa,7.302,1986-10-17
2,Four Rooms,5.820,1995-12-09
3,Judgment Night,6.527,1993-10-15
4,Life in Loops (A Megacities RMX),7.500,2006-01-01
5,Sonntag im August,7.135,2004-09-02
